In [1]:
import ee
import geemap

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-geos505-ubrb-prep')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-geos505-ubrb-prep')
    
google_drive_path = '_ubrb_geos505'

# Define the HUCs
hucs = ['17050112', '17050111', '17050113']


/Users/lejoflores/GEOS-505-Fall2025/env/lib/python3.12/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Import the HUC dataset
huc_dataset = ee.FeatureCollection('USGS/WBD/2017/HUC08')

# Filter the HUC dataset to include the specified HUCs
upper_boise_hucs = huc_dataset.filter(ee.Filter.inList('huc8', hucs))

# Calculate the geometry that bounds all selected HUCs
upper_boise_geometry = upper_boise_hucs.geometry().bounds()

# Import the USGS NED dataset (30m DEM)
dem = ee.Image('USGS/3DEP/10m')

# Clip the DEM to the Upper Boise River basin boundary
clipped_dem = dem.clip(upper_boise_geometry)

In [3]:
# Aggregate the DEM to a 30m spatial resolution using mean aggregation.
# Note that this will require resampling which may introduce some error.
aggregated_dem = clipped_dem.reduceResolution(
    reducer=ee.Reducer.mean(),
    maxPixels=1024
).reproject(
    crs=dem.projection().crs(),
    scale=30
)

# Export the image to Google Drive
task_config = {
    'folder': google_drive_path,  # Replace with desired folder name
    'region': upper_boise_geometry,
    'scale': 30,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'ubrb_dem_30m',
    'maxPixels': 1e13  # Adjust as needed based on image size
}

task = ee.batch.Export.image.toDrive(aggregated_dem, **task_config)
task.start()

print('Export task started. Check your Google Drive for the results.')


Export task started. Check your Google Drive for the results.


In [4]:
# Import the NLCD land cover dataset
nlcd = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter the NLCD dataset to get the most recent image
nlcd_image = nlcd.filterDate('2019-01-01', '2019-12-31').first()

# Select the land cover band
land_cover = nlcd_image.select('landcover')

# Clip the land cover data to the upper Boise River basin boundary
upper_boise_landcover = land_cover.clip(upper_boise_geometry)

# Display the land cover map (optional)
Map = geemap.Map()
Map.centerObject(upper_boise_geometry, 10)
Map.addLayer(upper_boise_landcover, {}, "Land Cover")
Map


Map(center=[43.66201770206228, -115.36830839214619], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
# Reproject the land cover to match the aggregated DEM
reprojected_landcover = upper_boise_landcover.reproject(
    crs=aggregated_dem.projection(),
    scale=30
)

# Export the reprojected land cover to Google Drive
task_config_reprojected_lc = {
    'folder': google_drive_path,
    'region': upper_boise_geometry,
    'scale': 30,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'upper_boise_landcover_30m',
    'maxPixels': 1e13
}

task_reprojected_lc = ee.batch.Export.image.toDrive(reprojected_landcover, **task_config_reprojected_lc)
task_reprojected_lc.start()

print('Reprojected land cover export task started. Check your Google Drive for the results.')

Reprojected land cover export task started. Check your Google Drive for the results.


In [6]:
# Export the geometry as a shapefile
task_config_shapefile = {
    'folder': google_drive_path,
    'fileNamePrefix': 'upper_boise_geometry',
    'fileFormat': 'SHP'
}

task_shapefile = ee.batch.Export.table.toDrive(upper_boise_hucs, **task_config_shapefile)
task_shapefile.start()

print('Shapefile export task started. Check your Google Drive for the results.')

Shapefile export task started. Check your Google Drive for the results.
